
# BBC News Classification
<p>CSCA-5632 Unsupervised Algorithms in Machine Learning</p>
<p>University of Colorado Boulder</p>
<p>@author alme9155@colorado.edu</p>
<hr/>

## Project Overview:
### This class project categorizes news articles using matrix factorization to predict the categories. 

This notebook includes the following:
* Brief EDA procedures
* Matrix factorization model building and training
* Model predictions and results
* Prediction results comparison with Supervised Learning

### Description
This project will use a public dataset from the BBC comprised of 2225 articles, each labeled under one of the five categories: business, entertainment, politics, sport, or tech. It is a balanced dataset that. Each category has about 400~500 articles, which ensures meaningful classification.

[@misc{learn-ai-bbc,
    author = {Bijoy Bose},
    title = {BBC News Classification},
    year = {2019},
    howpublished = {\url{https://kaggle.com/competitions/learn-ai-bbc}},
    note = {Kaggle}
}](https://kaggle.com/competitions/learn-ai-bbc})

In [1]:
from collections import Counter
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import spacy
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

